In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 25.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=9edcec08141d58d7ae5216bdf2d7725444f9f12eed2e42e6a1598d7b87d84ff8
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


# import necessary library

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
import random

spark = SparkSession.builder.appName('Recommendation system').getOrCreate()

In [ ]:
# Read the dataset
data = spark.read.csv('/content/drive/MyDrive/RS/ml-latest-small/ratings.csv',inferSchema=True,header=True)


In [ ]:
print(data.count(),len(data.columns))


100836 4


In [ ]:
data.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
data.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 10 rows



In [ ]:
data.groupBy('userId').count().orderBy('count',ascending=False).show(10,False)

+------+-----+
|userId|count|
+------+-----+
|414   |2698 |
|599   |2478 |
|474   |2108 |
|448   |1864 |
|274   |1346 |
|610   |1302 |
|68    |1260 |
|380   |1218 |
|606   |1115 |
|288   |1055 |
+------+-----+
only showing top 10 rows



In [ ]:
data.groupBy('userId').count().orderBy('count',ascending=True).show(10,False)

+------+-----+
|userId|count|
+------+-----+
|53    |20   |
|406   |20   |
|431   |20   |
|442   |20   |
|189   |20   |
|207   |20   |
|257   |20   |
|278   |20   |
|194   |20   |
|569   |20   |
+------+-----+
only showing top 10 rows



In [ ]:
data.groupBy('movieId').count().orderBy('count',ascending=False).show(10,False)

+-------+-----+
|movieId|count|
+-------+-----+
|356    |329  |
|318    |317  |
|296    |307  |
|593    |279  |
|2571   |278  |
|260    |251  |
|480    |238  |
|110    |237  |
|589    |224  |
|527    |220  |
+-------+-----+
only showing top 10 rows



In [ ]:
data.groupBy('rating').count().orderBy('count',ascending=False).show(10,False)

+------+-----+
|rating|count|
+------+-----+
|4.0   |26818|
|3.0   |20047|
|5.0   |13211|
|3.5   |13136|
|4.5   |8551 |
|2.0   |7551 |
|2.5   |5550 |
|1.0   |2811 |
|1.5   |1791 |
|0.5   |1370 |
+------+-----+



In [ ]:
#feature engineering
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer,IndexToString

In [ ]:
stringIndexer = StringIndexer(inputCol="movieId",outputCol="movieId_new")
model = stringIndexer.fit(data)

In [ ]:
indexed = model.transform(data)

In [ ]:
indexed.show(10)

+------+-------+------+---------+-----------+
|userId|movieId|rating|timestamp|movieId_new|
+------+-------+------+---------+-----------+
|     1|      1|   4.0|964982703|       11.0|
|     1|      3|   4.0|964981247|      422.0|
|     1|      6|   4.0|964982224|      129.0|
|     1|     47|   5.0|964983815|       15.0|
|     1|     50|   5.0|964982931|       14.0|
|     1|     70|   3.0|964982400|      390.0|
|     1|    101|   5.0|964980868|     1095.0|
|     1|    110|   4.0|964982176|        7.0|
|     1|    151|   5.0|964984041|      534.0|
|     1|    157|   5.0|964984100|     1991.0|
+------+-------+------+---------+-----------+
only showing top 10 rows



In [ ]:
indexed.groupBy('movieId_new').count().orderBy('count', ascending=False).show(10,False)

+-----------+-----+
|movieId_new|count|
+-----------+-----+
|0.0        |329  |
|1.0        |317  |
|2.0        |307  |
|3.0        |279  |
|4.0        |278  |
|5.0        |251  |
|6.0        |238  |
|7.0        |237  |
|8.0        |224  |
|9.0        |220  |
+-----------+-----+
only showing top 10 rows



In [ ]:
#spliting the dataset

train, test = indexed.randomSplit([0.75,0.25])

In [ ]:
train.count()

75597

In [ ]:
test.count()

25239

In [ ]:
#build and train recommeder model
from pyspark.ml.recommendation import ALS

rec = ALS(maxIter = 10, regParam=0.01,userCol='userId',itemCol='movieId_new',ratingCol='rating',nonnegative=True, coldStartStrategy="drop")


In [ ]:
rec_model = rec.fit(train)

# predictions and evalution on test data

In [ ]:
predicted_rating = rec_model.transform(test)

In [ ]:
predicted_rating.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- movieId_new: double (nullable = false)
 |-- prediction: float (nullable = false)



In [ ]:
predicted_rating.orderBy(rand()).show(10)

+------+-------+------+----------+-----------+----------+
|userId|movieId|rating| timestamp|movieId_new|prediction|
+------+-------+------+----------+-----------+----------+
|   288|  33794|   4.5|1145817438|       93.0| 4.0262127|
|    19|   1376|   4.0| 965703698|      552.0|  3.312953|
|   474|   4019|   4.0| 994819034|      921.0| 2.9155598|
|   140|   1608|   3.0| 949667202|      351.0| 3.1715355|
|   414|   2949|   3.0| 961515684|      787.0| 4.0705953|
|    80|   4720|   4.0|1377308974|      387.0|  4.319933|
|   597|   2100|   4.0| 941641110|      432.0| 3.7349045|
|   113|   1014|   5.0| 980307201|     3271.0|  3.591731|
|   474|    921|   3.0|1087832282|     2433.0| 3.1492658|
|   363|    260|   2.0|1502584358|        5.0| 2.5852103|
+------+-------+------+----------+-----------+----------+
only showing top 10 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse', predictionCol='prediction',labelCol='rating')

In [ ]:
rmse = evaluator.evaluate(predicted_rating)

In [ ]:
print('rmse',rmse)

rmse 1.0319759129196613


In [ ]:
#recommend top movie that active user might like
unique_movies_id = indexed.select('movieId_new').distinct()

In [ ]:
unique_movies_id.count()

9724

In [ ]:
a = unique_movies_id.alias('a')
a.columns

['movieId_new']

In [ ]:
#input-userid
user_id = 80

In [ ]:
#watched_movie
watched_movies = indexed.filter(indexed['userId'] == user_id).select('movieid_new').distinct()

In [ ]:
watched_movies.count()

167

In [ ]:
b = watched_movies.alias('b')
b.columns

['movieid_new']

In [ ]:
total_movies = a.join(b,a.movieid_new == b.movieid_new,how='left')

AttributeError: ignored